In [1]:
def unpack_results(OUTPUT_PATH):
    
    from os import listdir
    from os.path import isfile, join
    import pickle as pkl
    
    file_names = [f for f in listdir(OUTPUT_PATH) if isfile(join(OUTPUT_PATH, f))]
    data = dict()
    
    for file in file_names:
        if ".pkl" in file:
            f = pkl.load(open(OUTPUT_PATH + file, "rb"))
            data[file[:-4]] = f

    return data

In [2]:
def process_network_for_plots(G, number_of_layers, S):
    
    import numpy as np
    
    edges_layers = {layer: list() for layer in range(number_of_layers+1)}
    edges_layers_index = {layer: list() for layer in range(number_of_layers+1)}

    for n, edge in enumerate(G.edges()):

        # Synthetic data.
        layeri = G.nodes[edge[0]]["original_label"][1]
        layerj = G.nodes[edge[1]]["original_label"][1]
        if layeri == layerj:
            edges_layers[layeri].append(edge)
            edges_layers_index[layeri].append(n)
        else:
            edges_layers[number_of_layers].append(edge)
            edges_layers_index[number_of_layers].append(n)

    node_layers = {layer: list() for layer in range(number_of_layers)}
    for n, node in enumerate(G.nodes()):

        # Synthetic data.
        layeri = G.nodes[node]["original_label"][1]
        node_layers[layeri].append(node)

    size_node_layers = {layer: np.zeros(G.number_of_nodes()) for layer in range(number_of_layers)}

    for layer in range(number_of_layers):
        nodes = node_layers[layer]
        for node in nodes:
            size_node_layers[layer] += abs(S[:,node])

    return edges_layers, edges_layers_index, node_layers, size_node_layers

In [3]:
def plot_network_F(G, node_layers, size_node_layers, number_of_layers, edges_layers, edges_layers_index, F):

    import networkx as nx
    import matplotlib.pyplot as plt
    from matplotlib.patches import Patch

    prop_coeff_nodes = 30
    prop_coeff_edges = 3
    prop_coeff_edges_inter = 10

    num_connections = number_of_layers + int(number_of_layers-1)
    fig, ax = plt.subplots(1, 1, figsize=(5,5))

    pos = {node: G.nodes[node]["pos"] for node in G.nodes()}

    for layer, connection in enumerate(range(num_connections)[::2]):
        nx.draw_networkx_nodes(G, pos=pos, nodelist=node_layers[layer], node_size=2, node_color="black", ax=ax)

        flux_index = edges_layers_index[layer]
        nx.draw_networkx_edges(G, pos=pos, edgelist=edges_layers[layer], width=F[flux_index]*prop_coeff_edges, edge_color="C"+str(layer), ax=ax)

        # Topology.
        nx.draw_networkx_edges(G, pos=pos, edgelist=edges_layers[layer], width=1, edge_color="silver", ax=ax)

    inter_layer_edges = edges_layers[number_of_layers]

    for layer, connections in enumerate(list(zip(range(num_connections)[::2], range(num_connections)[::2][1:]))):

        layer1 = layer
        layer2 = layer + 1
        nodes1 = node_layers[layer1]
        nodes2 = node_layers[layer2]
        shared_edges = [e for e in G.edges() if (e[0] in nodes1 and e[1] in nodes2) or (e[0] in nodes2 and e[1] in nodes1)]
        shared_edges_index = [n for n, e in enumerate(G.edges()) if (e[0] in nodes1 and e[1] in nodes2) or (e[0] in nodes2 and e[1] in nodes1)]
        inter_nodes = [e[0] for e in shared_edges]

        index_plot = connections[1] - 1

        nx.draw_networkx_nodes(G, pos=pos, nodelist=inter_nodes, node_size=F[shared_edges_index]*prop_coeff_edges_inter, node_color="magenta", ax=ax)
        nx.draw_networkx_nodes(G, pos=pos, nodelist=inter_nodes, node_size=2, node_color="black", ax=ax)

    legend_elements = [Patch(facecolor='C0', edgecolor='None', lw=2, hatch="", label=r'Fluxes $k=1$, large layer'),
                       Patch(facecolor='C1', edgecolor='None', lw=2, hatch="", label=r'Fluxes $k=2$, small layer'),
                       Patch(facecolor='magenta', edgecolor='None', lw=2, hatch="", label=r'Inter-layer fluxes')]

    ax.legend(handles=legend_elements, loc="best",
    fancybox=False, framealpha=1.0, facecolor="None", edgecolor="None",
    fontsize=11, borderpad=0.1, labelspacing=0.05, borderaxespad=0.1,ncol=1,columnspacing=0, handlelength=2, handletextpad=0.5)

    plt.show()

In [4]:
def costs(data):

    try:
        J_dyn = data["results"]["J_dyn"]
        J_dijkstra = data["results"]["J_dijkstra"]
        J_dyn_sp = data["results"]["J_dyn_sp"]
        J_sp_dyn = data["results"]["J_sp_dyn"]
        J_sp_dijkstra = data["results"]["J_sp_dijkstra"]
        J_sp_dyn_sp = data["results"]["J_sp_dyn_sp"]
    except:
        pass

    import matplotlib.pyplot as plt
    from matplotlib.patches import Patch

    fig, ax = plt.subplots(1, 1, figsize=(5,3))
    plt.scatter([1,2,3], [J_dyn, J_dyn_sp, J_dijkstra], marker="s", c=["blue", "red", "green"], s=150)
    plt.xticks([1,2,3])
    plt.ylabel(r"$J(\beta)$, Multicommodity cost", size=13)

    legend_elements = [Patch(facecolor='blue', edgecolor='None', lw=2, hatch="", label=r'Multicommodity dynamics'),
                       Patch(facecolor='red', edgecolor='None', lw=2, hatch="", label=r'Shortest path dynamics'),
                       Patch(facecolor='green', edgecolor='None', lw=2, hatch="", label=r'Dijkstra')]

    ax.legend(handles=legend_elements, loc="best",
    fancybox=False, framealpha=1.0, facecolor="None", edgecolor="None",
    fontsize=11, borderpad=0.1, labelspacing=0.05, borderaxespad=0.1,ncol=1,columnspacing=0, handlelength=2, handletextpad=0.5)


    plt.show()

    fig, ax = plt.subplots(1, 1, figsize=(5,3))
    plt.scatter([1,2,3], [J_sp_dyn, J_sp_dijkstra, J_sp_dyn_sp], c=["blue", "red", "green"], marker="s", s=150)
    plt.xticks([1,2,3])
    plt.ylabel(r"$J$, Shortest path cost", size=13)
    plt.show()

    print("J_dyn:", J_dyn)
    print("J_dikstra:", J_dijkstra)
    print("J_dyn_sp:", J_dyn_sp)

    print("J_sp_dyn:", J_sp_dyn)
    print("J_sp_dijkstra:", J_sp_dijkstra)
    print("J_sp_dyn_sp:", J_sp_dyn_sp)